## Module Imports

In [7]:
import os
import torch
from torch import nn
from torch.utils.data import Dataset
from torchvision import transforms
import torchvision
from PIL import Image
import random
import numpy as np
import glob

## Device Configuration

In [8]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Defining the Model using PyTorch

In [9]:
# Model
class AODNet(nn.Module):
    def __init__(self):
        super(AODNet, self).__init__()
        self.b = nn.Parameter(torch.scalar_tensor(1))

        self.conv1 = nn.Conv2d(3, 3, kernel_size=1, padding=0, stride=1)
        self.conv2 = nn.Conv2d(3, 3, kernel_size=3, padding=1, stride=1)
        self.conv3 = nn.Conv2d(6, 3, kernel_size=5, padding=2, stride=1)
        self.conv4 = nn.Conv2d(6, 3, kernel_size=7, padding=3, stride=1)
        self.conv5 = nn.Conv2d(12, 3, kernel_size=3, padding=1, stride=1)

# The AOD-Net consists of five
# "concatl " layer of AOD-Net concatenates features from the layers "convl " and "conv2 ". Similarly, "concat2 " concatenates those from "conv2" and "conv3 "; "concat3" concatenates those from "convl", "conv2", "conv3", and "conv4". The kernel "K" is learned from the concatenated features. The output of the AOD-Net is the weighted sum of the input image and the kernel "K" minus "K" itself.

    def forward(self, x):
        x1 =  nn.functional.relu(self.conv1(x))
        x2 =  nn.functional.relu(self.conv2(x1))
        concat1 = torch.cat((x1, x2), 1)
        x3 = nn.functional.relu(self.conv3(concat1))
        concat2 = torch.cat((x2, x3), 1)
        x4 = nn.functional.relu(self.conv4(concat2))
        concat3 = torch.cat((x1, x2, x3, x4), 1)
        K = nn.functional.relu(self.conv5(concat3))

        output = torch.mul(K, x) - K + self.b
        return output
    
# Loss Function
loss_fn = nn.MSELoss()

## Dataset Configuration

In [18]:
# Dataset configuration
def create_training_set(original_img_data_path, hazy_img_data_path):
    train_list = []
    val_list = []
    
    hazy_img_list = glob.glob(hazy_img_data_path + '*.jpg')

    d = {}
    for img in hazy_img_list:
        i = img.split('\\')[-1]
        key = i.split('_')[0] + '_' + i.split('_')[1] + ".jpg"
        if key in d.keys():
            d[key].append(img.split('\\')[-1])
        else:
            d[key] = [img.split('\\')[-1]]
    
    train_imgs = []
    val_imgs = []

    for i in range(len(d.keys())):
        if i < 0.9 * len(d.keys()):
            train_imgs.append(list(d.keys())[i])
        else:
            val_imgs.append(list(d.keys())[i])

    for key in list(d.keys()):
        if key in train_imgs:
            for hazy_img in d[key]:
                train_list.append((original_img_data_path + key, hazy_img_data_path + hazy_img))
        else:
            for hazy_img in d[key]:
                val_list.append((original_img_data_path + key, hazy_img_data_path + hazy_img))
    random.shuffle(train_list)
    random.shuffle(val_list)
    return train_list, val_list

class create_dataloader(Dataset):
    def __init__(self, org_img_path, hazy_img_path, mode="train"):
        self.train_list, self.val_list = create_training_set(org_img_path, hazy_img_path)
        if mode == "train":
            self.data_list = self.train_list
            print('Total training images:', len(self.data_list))
        else:
            self.data_list = self.val_list
            print('Total validation images:', len(self.data_list))
    
    def transform(self, img):
        return torch.from_numpy(np.asarray(img.resize((480, 640), Image.Resampling.BILINEAR))).permute(2, 0, 1).float() / 255.0
    
    def __getitem__(self, idx):
        org_img_path, hazy_img_path = self.data_list[idx]
        org_img = Image.open(org_img_path)
        hazy_img = Image.open(hazy_img_path)
        org_img = self.transform(org_img)
        hazy_img = self.transform(hazy_img)
        return org_img, hazy_img
    
    def __len__(self): 
        return len(self.data_list)

In [ ]:
# Training the model
def train(config):
    def weights_init(m):
        classname = m.__class__.__name__
        if classname.find('Conv') != -1:
            nn.init.normal_(m.weight, 0.0, 0.02)
        elif classname.find('BatchNorm') != -1:
            nn.init.normal_(m.weight, 1.0, 0.02)
            nn.init.constant_(m.bias, 0)

    # Hyperparameters
    num_epochs = config['num_epochs']
    batch_size = config['batch_size']
    learning_rate = config['learning_rate']
    weight_decay = config['weight_decay']
    grad_clip_norm = config['grad_clip_norm']

    # Data loader
    train_dataset = create_dataloader(config['original_img_data_path'], config['hazy_img_data_path'], mode="train")
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

    val_dataset = create_dataloader(config['original_img_data_path'], config['hazy_img_data_path'], mode="val")
    val_loader = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)

    # Model
    model = AODNet()
    model.apply(weights_init)
    model.to(device)

    # Optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    model.train()

    # Train the model
    total_step = len(train_loader)
    for epoch in range(num_epochs):
        for i, (org_img, hazy_img) in enumerate(train_loader):
            org_img = org_img.to(device)
            hazy_img = hazy_img.to(device)
            
            clean_img = model(hazy_img)
            loss = loss_fn(clean_img, org_img)

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip_norm)
            optimizer.step()

            if (i+1) % config['display_iter'] == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            if (i+1) % config['save_iter'] == 0:
                torch.save(model.state_dict(), config['model_path'] + 'model-{}.ckpt'.format(epoch+1))

        # Validation
        for i, (org_img, hazy_img) in enumerate(val_loader):
            org_img = org_img.to(device)
            hazy_img = hazy_img.to(device)

            clean_img = model(hazy_img)
            loss = loss_fn(clean_img, org_img)

            torchvision.utils.save_image(torch.cat((hazy_img, clean_img, org_img),0), config['output_path'] + 'clean_img_{}.jpg'.format(i))
        torch.save(model.state_dict(), 'model-{}.ckpt'.format(epoch+1))

## Config Files

In [123]:
config = {
    'original_img_data_path': 'data\\image\\',
    'hazy_img_data_path': 'data\\data\\',
    'model_path': '.\\model\\',
    'output_path': '.\\output\\',
    'num_epochs': 10,
    'batch_size': 8,
    'learning_rate': 0.0001,
    'weight_decay': 0.0001,
    'grad_clip_norm': 0.1,
    'display_iter': 100,
    'save_iter': 100
}
if not os.path.exists(config['model_path']):
    os.makedirs(config['model_path'])
if not os.path.exists(config['output_path']):
    os.makedirs(config['output_path'])
train(config)

Total training images: 24443
Total validation images: 2813
Epoch [1/10], Step [100/3056], Loss: 0.3082
Epoch [1/10], Step [200/3056], Loss: 0.1782
Epoch [1/10], Step [300/3056], Loss: 0.1302
Epoch [1/10], Step [400/3056], Loss: 0.1239
Epoch [1/10], Step [500/3056], Loss: 0.1300
Epoch [1/10], Step [600/3056], Loss: 0.1414
Epoch [1/10], Step [700/3056], Loss: 0.1413
Epoch [1/10], Step [800/3056], Loss: 0.1152
Epoch [1/10], Step [900/3056], Loss: 0.1102
Epoch [1/10], Step [1000/3056], Loss: 0.1447
Epoch [1/10], Step [1100/3056], Loss: 0.1147
Epoch [1/10], Step [1200/3056], Loss: 0.1232
Epoch [1/10], Step [1300/3056], Loss: 0.1242
Epoch [1/10], Step [1400/3056], Loss: 0.0877
Epoch [1/10], Step [1500/3056], Loss: 0.1023
Epoch [1/10], Step [1600/3056], Loss: 0.1150
Epoch [1/10], Step [1700/3056], Loss: 0.0776
Epoch [1/10], Step [1800/3056], Loss: 0.0872
Epoch [1/10], Step [1900/3056], Loss: 0.0905
Epoch [1/10], Step [2000/3056], Loss: 0.0695
Epoch [1/10], Step [2100/3056], Loss: 0.0717
Epoch

## Model Inference

In [20]:
config = {
    'original_img_data_path': 'data\\image\\',
    'hazy_img_data_path': 'data\\data\\',
    'model_path': '.\\model\\',
    'output_path': '.\\output\\',
    'num_epochs': 10,
    'batch_size': 8,
    'learning_rate': 0.0001,
    'weight_decay': 0.0001,
    'grad_clip_norm': 0.1,
    'display_iter': 100,
    'save_iter': 100
}

In [ ]:
model = AODNet()
model.load_state_dict(torch.load('./model/model-10.ckpt'), strict=False)
model.eval()
model.to(device)
val_dataset = create_dataloader(config['original_img_data_path'], config['hazy_img_data_path'], mode="val")
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=config['batch_size'], shuffle=False)
for i, (org_img, hazy_img) in enumerate(val_loader):
        org_img = org_img.to(device)
        hazy_img = hazy_img.to(device)
        with torch.no_grad():
                clean_img = model(hazy_img)
        loss = loss_fn(clean_img, org_img)
        torchvision.utils.save_image(torch.cat((hazy_img, clean_img, org_img),0), "./output/" + 'clean_img_{}.jpg'.format(i))

## For NH-Haze Dataset

In [42]:
import os
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import numpy as np
import torch
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim 

# Custom Dataset
class create_dataloader_output(Dataset):
    def __init__(self, img_folder, mode="train"):
        # Create lists of hazy and ground truth images dynamically
        self.hazy_images = [os.path.join(img_folder, f) for f in os.listdir(img_folder) if not f.endswith('_GT.png')]
        self.gt_images = [os.path.join(img_folder, f) for f in os.listdir(img_folder) if f.endswith('_GT.png')]

        # Match hazy images with their corresponding ground truth
        self.data_list = []
        for hazy_img in self.hazy_images:
            gt_img = hazy_img.replace('_hazy.png', '_GT.png')  # Assuming same name with _GT
            if gt_img in self.gt_images:
                self.data_list.append((hazy_img, gt_img))

        print(f'Total images: {len(self.data_list)}')

    def transform(self, img):
        np_array = np.asarray(img.resize((480, 640), Image.BILINEAR)).copy()
        return torch.from_numpy(np_array).permute(2, 0, 1).float() / 255.0

    def __getitem__(self, idx):
        hazy_img_path, gt_img_path = self.data_list[idx]
        hazy_img = Image.open(hazy_img_path).convert("RGB")
        gt_img = Image.open(gt_img_path).convert("RGB")
        hazy_img = self.transform(hazy_img)
        gt_img = self.transform(gt_img)
        return hazy_img, gt_img

    def __len__(self):
        return len(self.data_list)


# Path to images
img_folder = "./NH-HAZE"
output_dir = "outputs"
os.makedirs(output_dir, exist_ok=True)

# Create Dataset and DataLoader
dataset = create_dataloader_output(img_folder)
data_loader = DataLoader(dataset, batch_size=5, shuffle=False)

# Assuming a pre-trained dehazing model is loaded
class DehazingModel(torch.nn.Module):
    def forward(self, x):
        return x * 0.9  # Dummy dehazing; replace with actual model logic

model = AODNet()
# weights_path = "./model/model-10.ckpt"  # Update with the correct path
model.load_state_dict(torch.load('./model/model-10.ckpt'), strict=True)
model.to(device)
model.eval()

total_psnr = 0
total_ssim = 0
num_images = 0

# Process and Save Batches
for idx, (hazy_imgs, gt_imgs) in enumerate(data_loader):
    with torch.no_grad():
        hazy_imgs = hazy_imgs.to(device)
        dehazed_imgs = model(hazy_imgs)  # Dehaze the hazy images

    

    # Plot and Save Results
    fig, axs = plt.subplots(5, 3, figsize=(7, 15))
    axs[0, 0].set_title("Hazy Image")
    axs[0, 1].set_title("Dehazed Image")
    axs[0, 2].set_title("Ground Truth")
    for i in range(len(hazy_imgs)):
        hazy_img_np = hazy_imgs[i].cpu().permute(1, 2, 0).numpy()
        dehazed_img_np = dehazed_imgs[i].cpu().permute(1, 2, 0).numpy()
        gt_img_np = gt_imgs[i].cpu().permute(1, 2, 0).numpy()
        hazy_img_np = np.clip(hazy_img_np, 0, 1)
        dehazed_img_np = np.clip(dehazed_img_np, 0, 1)
        gt_img_np = np.clip(gt_img_np, 0, 1)
        psnr_value = psnr(gt_img_np, dehazed_img_np, data_range=1.0)  # data_range=1.0 for normalized images
        ssim_value = ssim(gt_img_np, dehazed_img_np, multichannel=True, data_range=1.0, win_size=3)
        # Accumulate metrics
        total_psnr += psnr_value
        total_ssim += ssim_value
        num_images += 1
        axs[i, 0].imshow(hazy_img_np)
        axs[i, 1].imshow(dehazed_img_np)
        axs[i, 2].imshow(gt_img_np)
        axs[i, 0].axis("off")
        axs[i, 1].axis("off")
        axs[i, 2].axis("off")
        print(f"Image {num_images}: PSNR = {psnr_value:.2f}, SSIM = {ssim_value:.4f}")
    plt.tight_layout()

    # Save the visualization to the outputs folder
    save_path = os.path.join(output_dir, f"batch_{idx + 1}.png")
    plt.savefig(save_path)
    plt.close(fig)

    print(f"Saved batch {idx + 1} visualization to {save_path}")


# Calculate averages
average_psnr = total_psnr / num_images
average_ssim = total_ssim / num_images

# Print final results
print(f"Average PSNR: {average_psnr:.2f}")
print(f"Average SSIM: {average_ssim:.4f}")


Total images: 55


C:\Users\rahul chauhan\AppData\Local\Temp\ipykernel_37400\3880783649.py:58: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./model/model-10.

Image 1: PSNR = 14.69, SSIM = 0.1861
Image 2: PSNR = 14.38, SSIM = 0.4413
Image 3: PSNR = 11.20, SSIM = 0.3630
Image 4: PSNR = 10.80, SSIM = 0.5842
Image 5: PSNR = 12.39, SSIM = 0.4228
Saved batch 1 visualization to outputs\batch_1.png
Image 6: PSNR = 9.82, SSIM = 0.1767
Image 7: PSNR = 10.13, SSIM = 0.1963
Image 8: PSNR = 10.63, SSIM = 0.3962
Image 9: PSNR = 9.81, SSIM = 0.2107
Image 10: PSNR = 12.82, SSIM = 0.2301
Saved batch 2 visualization to outputs\batch_2.png
Image 11: PSNR = 13.24, SSIM = -0.1122
Image 12: PSNR = 13.15, SSIM = -0.0012
Image 13: PSNR = 11.67, SSIM = 0.2360
Image 14: PSNR = 10.77, SSIM = 0.3951
Image 15: PSNR = 15.07, SSIM = 0.5516
Saved batch 3 visualization to outputs\batch_3.png
Image 16: PSNR = 12.43, SSIM = 0.5196
Image 17: PSNR = 13.54, SSIM = 0.1181
Image 18: PSNR = 13.02, SSIM = 0.2327
Image 19: PSNR = 12.71, SSIM = 0.0356
Image 20: PSNR = 12.13, SSIM = -0.0980
Saved batch 4 visualization to outputs\batch_4.png
Image 21: PSNR = 11.50, SSIM = 0.1708
Image 

## For SOTs Indoor

In [47]:
import os
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import numpy as np
import torch
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim 

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset Class
class PairedImageDataset(Dataset):
    def __init__(self, root_folder, transform=None):
        """
        Args:
            root_folder (str): Path to the dataset folder containing 'clear' and 'hazy' subfolders.
            transform (callable, optional): Transform to apply to the images.
        """
        self.clear_folder = os.path.join(root_folder, "clear")
        self.hazy_folder = os.path.join(root_folder, "hazy")
        
        self.hazy_images = sorted([f for f in os.listdir(self.hazy_folder) if f.endswith('_5.png')])
        self.clear_images = [f.replace('_5', '') for f in self.hazy_images]
        
        for clear_img in self.clear_images:
            if not os.path.exists(os.path.join(self.clear_folder, clear_img)):
                raise FileNotFoundError(f"Missing clear image: {clear_img}")
        
        if transform is None:
            self.transform = transforms.Compose([
                transforms.Resize((256, 256)),  
                transforms.ToTensor(),        
            ])
        else:
            self.transform = transform

    def __getitem__(self, idx):
        hazy_path = os.path.join(self.hazy_folder, self.hazy_images[idx])
        clear_path = os.path.join(self.clear_folder, self.clear_images[idx])
        
        hazy_img = Image.open(hazy_path).convert("RGB")
        clear_img = Image.open(clear_path).convert("RGB")
        
        hazy_img = self.transform(hazy_img)
        clear_img = self.transform(clear_img)
        
        return hazy_img, clear_img

    def __len__(self):
        return len(self.hazy_images)

# Dehazing Model (Dummy Example)
class DehazingModel(torch.nn.Module):
    def forward(self, x):
        return x * 0.9  

# Path setup
root_folder = "./indoor"  
output_dir = "outputs_SOT"
os.makedirs(output_dir, exist_ok=True)

# Initialize dataset and dataloader
dataset = PairedImageDataset(root_folder)
data_loader = DataLoader(dataset, batch_size=5, shuffle=False)

# Load Model
model = AODNet()  
weights_path = "./model/model-10.ckpt"
model.load_state_dict(torch.load(weights_path, map_location=device), strict=True)
model.to(device)
model.eval()

# Metrics Initialization
total_psnr = 0
total_ssim = 0
num_images = 0

# Process and Evaluate
for idx, (hazy_imgs, clear_imgs) in enumerate(data_loader):
    hazy_imgs = hazy_imgs.to(device)
    clear_imgs = clear_imgs.to(device)

    with torch.no_grad():
        dehazed_imgs = model(hazy_imgs)

    # Visualization and Metrics Calculation
    fig, axs = plt.subplots(5, 3, figsize=(8, 13))
    axs[0, 0].set_title("Hazy Image")
    axs[0, 1].set_title("Dehazed Image")
    axs[0, 2].set_title("Clear Image")

    for i in range(len(hazy_imgs)):
        # Convert tensors to numpy for visualization and metrics
        hazy_img_np = hazy_imgs[i].cpu().permute(1, 2, 0).numpy()
        dehazed_img_np = dehazed_imgs[i].cpu().permute(1, 2, 0).numpy()
        clear_img_np = clear_imgs[i].cpu().permute(1, 2, 0).numpy()

        # Clip values to valid range
        hazy_img_np = np.clip(hazy_img_np, 0, 1)
        dehazed_img_np = np.clip(dehazed_img_np, 0, 1)
        clear_img_np = np.clip(clear_img_np, 0, 1)

        # Compute PSNR and SSIM
        psnr_value = psnr(clear_img_np, dehazed_img_np, data_range=1.0)
        ssim_value = ssim(clear_img_np, dehazed_img_np, channel_axis=-1, data_range=1.0)

        total_psnr += psnr_value
        total_ssim += ssim_value
        num_images += 1

        # Plot images
        axs[i, 0].imshow(hazy_img_np)
        axs[i, 1].imshow(dehazed_img_np)
        axs[i, 2].imshow(clear_img_np)
        axs[i, 0].axis("off")
        axs[i, 1].axis("off")
        axs[i, 2].axis("off")

        print(f"Image {num_images}: PSNR = {psnr_value:.2f}, SSIM = {ssim_value:.4f}")

    plt.tight_layout()

    # Save visualization
    save_path = os.path.join(output_dir, f"batch_{idx + 1}.png")
    plt.savefig(save_path)
    plt.close(fig)

    print(f"Saved batch {idx + 1} visualization to {save_path}")

# Final Metrics
average_psnr = total_psnr / num_images
average_ssim = total_ssim / num_images
print(f"Average PSNR: {average_psnr:.2f}")
print(f"Average SSIM: {average_ssim:.4f}")

C:\Users\rahul chauhan\AppData\Local\Temp\ipykernel_37400\4286031947.py:74: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(weights_path, map_

Image 1: PSNR = 17.17, SSIM = 0.8234
Image 2: PSNR = 20.16, SSIM = 0.8224
Image 3: PSNR = 20.55, SSIM = 0.8014
Image 4: PSNR = 17.35, SSIM = 0.7251
Image 5: PSNR = 21.31, SSIM = 0.9063
Saved batch 1 visualization to outputs_SOT\batch_1.png
Image 6: PSNR = 20.67, SSIM = 0.9095
Image 7: PSNR = 21.16, SSIM = 0.9071
Image 8: PSNR = 20.88, SSIM = 0.9226
Image 9: PSNR = 20.26, SSIM = 0.9265
Image 10: PSNR = 21.19, SSIM = 0.9205
Saved batch 2 visualization to outputs_SOT\batch_2.png
Image 11: PSNR = 19.81, SSIM = 0.9053
Image 12: PSNR = 17.66, SSIM = 0.8059
Image 13: PSNR = 18.74, SSIM = 0.8503
Image 14: PSNR = 14.84, SSIM = 0.8101
Image 15: PSNR = 17.38, SSIM = 0.8418
Saved batch 3 visualization to outputs_SOT\batch_3.png
Image 16: PSNR = 20.90, SSIM = 0.8902
Image 17: PSNR = 18.30, SSIM = 0.8921
Image 18: PSNR = 19.68, SSIM = 0.9087
Image 19: PSNR = 20.53, SSIM = 0.8966
Image 20: PSNR = 19.97, SSIM = 0.9019
Saved batch 4 visualization to outputs_SOT\batch_4.png
Image 21: PSNR = 15.31, SSIM 

## For SOTs outdoor

In [51]:
import os
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import numpy as np
import torch
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim 

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset Class
class PairedImageDataset(Dataset):
    def __init__(self, root_folder, transform=None):
        """
        Args:
            root_folder (str): Path to the dataset folder containing 'clear' and 'hazy' subfolders.
            transform (callable, optional): Transform to apply to the images.
        """
        self.clear_folder = os.path.join(root_folder, "clear")
        self.hazy_folder = os.path.join(root_folder, "hazy")
        
        # Filter hazy images ending with '_5.png' and generate corresponding clear image names
        self.hazy_images = sorted([f for f in os.listdir(self.hazy_folder) if f.endswith('.jpg')])
        self.clear_images = [f"{f.split('_')[0]}.png" for f in self.hazy_images]
        
        for clear_img in self.clear_images:
            if not os.path.exists(os.path.join(self.clear_folder, clear_img)):
                raise FileNotFoundError(f"Missing clear image: {clear_img}")
        
        # Default transformations
        if transform is None:
            self.transform = transforms.Compose([
                transforms.Resize((256, 256)),  # Resize to fixed size
                transforms.ToTensor(),         # Convert to tensor
            ])
        else:
            self.transform = transform

    def __getitem__(self, idx):
        hazy_path = os.path.join(self.hazy_folder, self.hazy_images[idx])
        clear_path = os.path.join(self.clear_folder, self.clear_images[idx])
        
        hazy_img = Image.open(hazy_path).convert("RGB")
        clear_img = Image.open(clear_path).convert("RGB")
        
        hazy_img = self.transform(hazy_img)
        clear_img = self.transform(clear_img)
        
        return hazy_img, clear_img

    def __len__(self):
        return len(self.hazy_images)

# Dehazing Model (Dummy Example)
class DehazingModel(torch.nn.Module):
    def forward(self, x):
        return x * 0.9  # Replace with actual model logic

# Path setup
root_folder = "./outdoor"  # Dataset folder containing 'clear' and 'hazy'
output_dir = "outputs_SOT_2"
os.makedirs(output_dir, exist_ok=True)

# Initialize dataset and dataloader
dataset = PairedImageDataset(root_folder)
data_loader = DataLoader(dataset, batch_size=5, shuffle=False)

# Load Model
model = AODNet() 
weights_path = "./model/model-10.ckpt"
model.load_state_dict(torch.load(weights_path, map_location=device), strict=True)
model.to(device)
model.eval()

# Metrics Initialization
total_psnr = 0
total_ssim = 0
num_images = 0

# Process and Evaluate
for idx, (hazy_imgs, clear_imgs) in enumerate(data_loader):
    hazy_imgs = hazy_imgs.to(device)
    clear_imgs = clear_imgs.to(device)

    with torch.no_grad():
        dehazed_imgs = model(hazy_imgs)

    # Visualization and Metrics Calculation
    fig, axs = plt.subplots(5, 3, figsize=(8, 13))
    axs[0, 0].set_title("Hazy Image")
    axs[0, 1].set_title("Dehazed Image")
    axs[0, 2].set_title("Clear Image")

    for i in range(len(hazy_imgs)):
        # Convert tensors to numpy for visualization and metrics
        hazy_img_np = hazy_imgs[i].cpu().permute(1, 2, 0).numpy()
        dehazed_img_np = dehazed_imgs[i].cpu().permute(1, 2, 0).numpy()
        clear_img_np = clear_imgs[i].cpu().permute(1, 2, 0).numpy()

        # Clip values to valid range
        hazy_img_np = np.clip(hazy_img_np, 0, 1)
        dehazed_img_np = np.clip(dehazed_img_np, 0, 1)
        clear_img_np = np.clip(clear_img_np, 0, 1)

        # Compute PSNR and SSIM
        psnr_value = psnr(clear_img_np, dehazed_img_np, data_range=1.0)
        ssim_value = ssim(clear_img_np, dehazed_img_np, channel_axis=-1, data_range=1.0)

        total_psnr += psnr_value
        total_ssim += ssim_value
        num_images += 1

        # Plot images
        axs[i, 0].imshow(hazy_img_np)
        axs[i, 1].imshow(dehazed_img_np)
        axs[i, 2].imshow(clear_img_np)
        axs[i, 0].axis("off")
        axs[i, 1].axis("off")
        axs[i, 2].axis("off")

        print(f"Image {num_images}: PSNR = {psnr_value:.2f}, SSIM = {ssim_value:.4f}")

    plt.tight_layout()

    # Save visualization
    save_path = os.path.join(output_dir, f"batch_{idx + 1}.png")
    plt.savefig(save_path)
    plt.close(fig)

    print(f"Saved batch {idx + 1} visualization to {save_path}")

# Final Metrics
average_psnr = total_psnr / num_images
average_ssim = total_ssim / num_images
print(f"Average PSNR: {average_psnr:.2f}")
print(f"Average SSIM: {average_ssim:.4f}")

C:\Users\rahul chauhan\AppData\Local\Temp\ipykernel_37400\2540859786.py:74: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(weights_path, map_

Image 1: PSNR = 17.85, SSIM = 0.9175
Image 2: PSNR = 16.86, SSIM = 0.8215
Image 3: PSNR = 16.64, SSIM = 0.8915
Image 4: PSNR = 17.98, SSIM = 0.9389
Image 5: PSNR = 17.72, SSIM = 0.8617
Saved batch 1 visualization to outputs_SOT_2\batch_1.png
Image 6: PSNR = 22.07, SSIM = 0.9104
Image 7: PSNR = 18.95, SSIM = 0.8773
Image 8: PSNR = 17.56, SSIM = 0.8086
Image 9: PSNR = 21.14, SSIM = 0.8718
Image 10: PSNR = 19.00, SSIM = 0.9243
Saved batch 2 visualization to outputs_SOT_2\batch_2.png
Image 11: PSNR = 15.39, SSIM = 0.8298
Image 12: PSNR = 19.30, SSIM = 0.9122
Image 13: PSNR = 20.90, SSIM = 0.9266
Image 14: PSNR = 18.91, SSIM = 0.9152
Image 15: PSNR = 17.22, SSIM = 0.9018
Saved batch 3 visualization to outputs_SOT_2\batch_3.png
Image 16: PSNR = 17.01, SSIM = 0.8682
Image 17: PSNR = 20.95, SSIM = 0.9166
Image 18: PSNR = 20.00, SSIM = 0.8627
Image 19: PSNR = 18.85, SSIM = 0.9001
Image 20: PSNR = 20.61, SSIM = 0.8824
Saved batch 4 visualization to outputs_SOT_2\batch_4.png
Image 21: PSNR = 17.7

## For Smoke

In [63]:
import os
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import numpy as np
import torch
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim 

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset Class
class PairedImageDataset(Dataset):
    def __init__(self, root_folder, transform=None):
        """
        Args:
            root_folder (str): Path to the dataset folder containing 'clear' and 'hazy' subfolders.
            transform (callable, optional): Transform to apply to the images.
        """
    
        self.clear_folder = os.path.join(root_folder, "clean")
        self.hazy_folder = os.path.join(root_folder, "hazy")
        
        # Filter hazy images ending with '_5.png' and generate corresponding clear image names
        self.hazy_images = sorted([f for f in os.listdir(self.hazy_folder) if f.endswith('.png')])
        self.clear_images = [f"{f}" for f in self.hazy_images]
        
        for clear_img in self.clear_images:
            if not os.path.exists(os.path.join(self.clear_folder, clear_img)):
                raise FileNotFoundError(f"Missing clear image: {clear_img}")
        
        # Default transformations
        if transform is None:
            self.transform = transforms.Compose([
                transforms.Resize((256, 256)),  # Resize to fixed size
                transforms.ToTensor(),         # Convert to tensor
            ])
        else:
            self.transform = transform

    def __getitem__(self, idx):
        hazy_path = os.path.join(self.hazy_folder, self.hazy_images[idx])
        clear_path = os.path.join(self.clear_folder, self.clear_images[idx])
        
        hazy_img = Image.open(hazy_path).convert("RGB")
        clear_img = Image.open(clear_path).convert("RGB")
        
        hazy_img = self.transform(hazy_img)
        clear_img = self.transform(clear_img)
        
        return hazy_img, clear_img

    def __len__(self):
        return len(self.hazy_images)

# Dehazing Model (Dummy Example)
class DehazingModel(torch.nn.Module):
    def forward(self, x):
        return x * 0.9  # Replace with actual model logic

# Path setup
root_folder = "./Smoke"  # Dataset folder containing 'clear' and 'hazy'
output_dir = "outputs_Smoke_test"
os.makedirs(output_dir, exist_ok=True)

# Initialize dataset and dataloader
dataset = PairedImageDataset(root_folder)
data_loader = DataLoader(dataset, batch_size=5, shuffle=False)

# Load Model
model = AODNet() 
weights_path = "./model/model-10.ckpt"
model.load_state_dict(torch.load(weights_path, map_location=device), strict=True)
model.to(device)
model.eval()

# Metrics Initialization
total_psnr = 0
total_ssim = 0
num_images = 0

# Process and Evaluate
for idx, (hazy_imgs, clear_imgs) in enumerate(data_loader):
    hazy_imgs = hazy_imgs.to(device)
    clear_imgs = clear_imgs.to(device)

    with torch.no_grad():
        dehazed_imgs = model(hazy_imgs)

    # Visualization and Metrics Calculation
    fig, axs = plt.subplots(5, 3, figsize=(8, 13))
    axs[0, 0].set_title("Hazy Image")
    axs[0, 1].set_title("Dehazed Image")
    axs[0, 2].set_title("Clear Image")

    for i in range(len(hazy_imgs)):
        # Convert tensors to numpy for visualization and metrics
        hazy_img_np = hazy_imgs[i].cpu().permute(1, 2, 0).numpy()
        dehazed_img_np = dehazed_imgs[i].cpu().permute(1, 2, 0).numpy()
        clear_img_np = clear_imgs[i].cpu().permute(1, 2, 0).numpy()

        # Clip values to valid range
        hazy_img_np = np.clip(hazy_img_np, 0, 1)
        dehazed_img_np = np.clip(dehazed_img_np, 0, 1)
        clear_img_np = np.clip(clear_img_np, 0, 1)

        # Compute PSNR and SSIM
        psnr_value = psnr(clear_img_np, dehazed_img_np, data_range=1.0)
        ssim_value = ssim(clear_img_np, dehazed_img_np, channel_axis=-1, data_range=1.0)

        total_psnr += psnr_value
        total_ssim += ssim_value
        num_images += 1

        # Plot images
        axs[i, 0].imshow(hazy_img_np)
        axs[i, 1].imshow(dehazed_img_np)
        axs[i, 2].imshow(clear_img_np)
        axs[i, 0].axis("off")
        axs[i, 1].axis("off")
        axs[i, 2].axis("off")

        print(f"Image {num_images}: PSNR = {psnr_value:.2f}, SSIM = {ssim_value:.4f}")

    plt.tight_layout()

    # Save visualization
    save_path = os.path.join(output_dir, f"batch_{idx + 1}.png")
    plt.savefig(save_path)
    plt.close(fig)

    print(f"Saved batch {idx + 1} visualization to {save_path}")

# Final Metrics
average_psnr = total_psnr / num_images
average_ssim = total_ssim / num_images
print(f"Average PSNR: {average_psnr:.2f}")
print(f"Average SSIM: {average_ssim:.4f}")

C:\Users\rahul chauhan\AppData\Local\Temp\ipykernel_37400\380405263.py:75: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(weights_path, map_l

Image 1: PSNR = 11.61, SSIM = 0.4425
Image 2: PSNR = 11.74, SSIM = 0.3760
Image 3: PSNR = 13.47, SSIM = 0.4705
Image 4: PSNR = 12.28, SSIM = 0.4356
Image 5: PSNR = 11.09, SSIM = 0.3935
Saved batch 1 visualization to outputs_Smoke_test\batch_1.png
Image 6: PSNR = 8.88, SSIM = 0.0751
Image 7: PSNR = 11.65, SSIM = 0.4768
Image 8: PSNR = 11.04, SSIM = 0.4000
Image 9: PSNR = 12.61, SSIM = 0.4146
Image 10: PSNR = 12.36, SSIM = 0.3986
Saved batch 2 visualization to outputs_Smoke_test\batch_2.png
Image 11: PSNR = 11.99, SSIM = 0.2733
Image 12: PSNR = 12.10, SSIM = 0.4816
Saved batch 3 visualization to outputs_Smoke_test\batch_3.png
Average PSNR: 11.73
Average SSIM: 0.3865
